![BridgingAI Logo](../bridgingai_logo.png)

# Deep Learning - Exercise 2: Practical Deep Learning 
1. [Initialization](#chp1-initialization)
2. [Batch Normalization](#chp2-batch-normalization)
3. [Layer Normalization](#chp3-layer-normalization)
4. [Dropout](#chp4-dropout)
5. [Optimization](#chp5-optimization)
6. [Experiments](#chp6-experiments)
---

In [ ]:
import torch
import torch.nn as nn

import math
from dataclasses import replace
from pprint import pprint
from typing import Union, List, Iterable

from sanity_checks import run_tests
from training import ExperimentConfig, Trainer, create_dataloader
from helpers import analyze_activations_combined, InitMethods, train_init_methods, train_model, evaluate_model

<a id="chp1-initialization"></a>
# 1. Initialization

Proper initialization is a crucial step when training deep neural networks. A bad initialization will have detrimental impact on the convergence of the network.

#### Example: Xavier Initialization

[Xavier initialization](https://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf), also known as Glorot initialization, is designed to keep the scale of gradients roughly the same in all layers, _when using tanh activations_. To this end, we need to initialize the weights with a variance of $\frac{2}{n_{\text{in}} + n_{\text{out}}}$, where $n_{in}$ is the number of input neurons and $n_{out}$ is the number of output neurons. We may choose between a uniform or a Gaussian distribution, as long as the variance is correct.

Uniform:
$W \sim U(-\sqrt{\frac{6}{n_{in} + n_{out}}}, \sqrt{\frac{6}{n_{in} + n_{out}}})$

Gaussian:
$W \sim N(0, \frac{2}{n_{\text{in}} + n_{\text{out}}})$

For more initialization methods (e.g. for different activation functions), refer to the documentation on the [torch.nn.init](https://pytorch.org/docs/stable/nn.init.html) module.

**TODO**: Implement Xavier initialization in `custom_xavier_init` function and run the cell to see a visualization of the activation values in the first forward pass.

**HINT**: 
1. To create a uniform distribution between 0 and 1, you can use `torch.rand(...)`.
2. We also provide the offical PyTorch implementation of Xavier initialization for visualization. If implemented correctly, the two methods should have similar results.

In [ ]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(784, 256),
            nn.Tanh(),
            nn.Linear(256, 128),
            nn.Tanh(),
            nn.Linear(128, 64),
            nn.Tanh(),
            nn.Linear(64, 10),
        )

    def forward(self, x):
        x = x.view(-1, 784)
        return self.net(x)


def custom_xavier_init(m):
    if isinstance(m, nn.Linear):
        in_features = m.in_features  # int
        out_features = m.out_features  # int
        weight = m.weight.data  # torch.Tensor

        # YOUR CODE HERE
        raise NotImplementedError()

        m.weight.data = weight
        nn.init.zeros_(m.bias)


init_methods = {
    "Xavier (Yours)": custom_xavier_init,
    "Xavier (PyTorch)": InitMethods.xavier_init,
    "Xavier x0.1": lambda m: InitMethods.scaled_xavier_init(m, 0.1),
    "Xavier x10": lambda m: InitMethods.scaled_xavier_init(m, 10),
    "Zero": InitMethods.zero_init,
    "Unit Gaussian": InitMethods.unit_gaussian_init,
}

train_loader, test_loader = create_dataloader(ExperimentConfig("", ""))
analyze_activations_combined(init_methods, SimpleNet, train_loader)

We'll train a simple neural network using various initialization methods to compare their performance. Some methods are intentionally flawed, resulting in stalled loss or high initial loss. Which models perform well?

In [ ]:
train_init_methods(SimpleNet, init_methods, train_loader, test_loader, epochs=3)

<a id="chp2-batch-normalization"></a>
# 2. Batch Normalization

Batch normalization normalizes the input of each layer in order to improve the training speed and stability of the neural network. It transforms the input to have a mean of 0 and a standard deviation of 1. This helps to prevent the input from becoming too large or too small, which can cause the network to become unstable. In this part, you will implement the batch normalization layer in PyTorch.

**TODO**: Implement the `forward` method in `BatchNorm` class below.

**HINT**: 
1. This layer is similar to the `nn.BatchNorm1d` layer in PyTorch. You can refer to the [documentation](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html) for information like the parameters and the formula used. One difference is that our implentation only takes 2D input tensors, while the PyTorch implementation can take 2D and 3D.
2. After reading the documentation, take a look at the `__init__` method to see what parameters are created. You will also need to call the `update_stats` method in the `forward` method to update the running mean and variance.
3. Since the module behaves differently during training and evaluation, you will need to check if the module is in training mode or not. You can use the `self.training` flag to check this.

In [ ]:
class BatchNorm(nn.Module):
    def __init__(self, num_features, eps=1e-05, momentum=0.1, affine=True):
        """
        Batch Normalization layer for data shape (N, C)
        """
        super().__init__()
        self.num_features = num_features
        self.eps = eps
        self.momentum = momentum
        self.affine = affine

        # always track running statistics for inference
        self.running_mean = torch.zeros(self.num_features)
        self.running_var = torch.ones(self.num_features)

        self.gamma = None
        self.beta = None
        if self.affine is True:
            self.gamma = nn.Parameter(torch.ones(self.num_features))
            self.beta = nn.Parameter(torch.zeros(self.num_features))

    def update_stats(self, batch_mean, batch_var):
        assert batch_mean.shape == self.running_mean.shape
        assert batch_var.shape == self.running_var.shape

        self.running_mean = (
            1 - self.momentum
        ) * self.running_mean + self.momentum * batch_mean
        self.running_var = (
            1 - self.momentum
        ) * self.running_var + self.momentum * batch_var

    def forward(self, x):
        """
        Inputs:
            x: input tensor of shape (N, C)
        Returns:
            x: output tensor of shape (N, C)
        """
        assert x.ndim == 2 and x.shape[1] == self.num_features and x.shape[0] > 1
        if self.training:
            # Train mode
            # YOUR CODE HERE
            raise NotImplementedError()

            if self.affine:
                x = x * self.gamma + self.beta
        else:
            # Test mode
            # YOUR CODE HERE
            raise NotImplementedError()

            if self.affine:
                x = x * self.gamma + self.beta
        return x

In [ ]:
# Run the sanity check
run_tests(BatchNorm=BatchNorm)

Networks with normalization layers usually converge faster than those without. To test this, we will train a simple network with and without batch normalization layers.

In [ ]:
model = nn.Sequential(nn.Flatten(1), nn.Linear(784, 128), nn.Tanh(), nn.Linear(128, 10))
train_model(model, train_loader, epochs=1)
acc = evaluate_model(model, test_loader)
print(f"Accuracy w/o BatchNorm: {acc:.4f}")

print()
# If your implementation does not work, uncomment the following line
# BatchNorm = nn.BatchNorm1d
bn_model = nn.Sequential(nn.Flatten(1), nn.Linear(784, 128), nn.Tanh(), BatchNorm(128), nn.Linear(128, 10))
train_model(bn_model, train_loader, epochs=1)
acc = evaluate_model(bn_model, test_loader)
print(f"Accuracy w/  BatchNorm: {acc:.4f}")

<a id="chp3-layer-normalization"></a>
# 3. Layer Normalization

Although batch normalization is widely used in deep learning, it has some limitations. For example, it can be sensitive to the batch size and have to keep track of the running mean and variance during training. Layer normalization is an alternative normalization technique that normalizes the input of each layer across the features, rather than across the batch dimension. In this part, you will have to implement the layer normalization layer in PyTorch.

**TODO**: Implement the `forward` method in `LayerNorm` class below.

**HINT**:
1. This layer is similar to the `nn.LayerNorm` layer in PyTorch. You can refer to the [documentation](https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html) for information like the parameters and the formula used.
2. After reading the documentation, take a look at the `__init__` method to see what parameters are created.

In [ ]:
class LayerNorm(nn.Module):
    def __init__(self, normalized_shape, eps=1e-05, elementwise_affine=True):
        """
        Layer Normalization layer

        Inputs:
        - normalized_shape: (int or list or torch.Size): input shape from an expected input of size (*, normalized_shape[0], normalized_shape[1], ..., normalized_shape[-1])
        - eps (float): a value added to the denominator for numerical stability. Default: 1e-5
        - elementwise_affine (bool): a boolean value that when set to True, this module has learnable per-element affine parameters. Default: True
        """
        super().__init__()

        if isinstance(normalized_shape, int):
            normalized_shape = (normalized_shape,)

        self.normalized_shape = torch.Size(normalized_shape)
        self.eps = eps
        self.elementwise_affine = elementwise_affine

        if self.elementwise_affine:
            self.weight = nn.Parameter(torch.ones(normalized_shape))
            self.bias = nn.Parameter(torch.zeros(normalized_shape))
        else:
            self.register_parameter("weight", None)
            self.register_parameter("bias", None)

    def forward(self, x):
        """
        Inputs:
        x: input tensor of shape (*, normalized_shape[0], normalized_shape[1], ..., normalized_shape[-1])

        Returns:
        output tensor of shape (*, normalized_shape[0], normalized_shape[1], ..., normalized_shape[-1])
        """
        # YOUR CODE HERE
        raise NotImplementedError()

In [ ]:
run_tests(LayerNorm=LayerNorm)

Layernorm often achieves similar or better results than BatchNorm, and it is less sensitive to the batch size. Run the following code to check the accuracy of the model with LayerNorm.

In [ ]:
bn_model = nn.Sequential(nn.Flatten(1), nn.Linear(784, 128), nn.Tanh(), LayerNorm(128), nn.Linear(128, 10))
train_model(bn_model, train_loader, epochs=1)
acc = evaluate_model(bn_model, test_loader)
print(f"Accuracy w/ LayerNorm: {acc:.4f}")

<a id="chp4-dropout"></a>
# 4. Dropout

[Dropout](https://www.cs.toronto.edu/~rsalakhu/papers/srivastava14a.pdf) is a regularization technique used to prevent overfitting in neural networks. It works by randomly setting a fraction of the input units to zero during training. This helps to prevent the network from becoming too dependent on any one input unit, and encourages the network to learn more robust features. In this part, you will implement the dropout layer in PyTorch.

**Vanilla Dropout Algorithm:**
1. During training, for each input x:
   y = x * mask, where mask ~ Bernoulli(1 - p)
2. During inference:
   y = (1 - p) * x

**Inverted Dropout Algorithm:**
1. During training, for each input x:
   y = (x * mask) / (1 - p), where mask ~ Bernoulli(1 - p)
2. During inference:
   y = x

The rescaling of activations is necessary to maintain the expected variance of activations. The original and inverted formulation are mathematically equivalent, but inverted dropout has the benefit of simplified inference.

**TODO**: Implement the `forward` method in the `Dropout` class below. Please implement **inverted dropout**, where scaling happens during training instead of testing. This is a common approach in modern deep learning frameworks.


In [ ]:
class Dropout(nn.Module):
    def __init__(self, p):
        super().__init__()
        self.p = p

    def forward(self, x):
        """
        Inputs:
            x: input tensor of shape (N, C)
        Returns:
            x: output tensor of shape (N, C)
        """
        if self.training:
            # Train mode
            # YOUR CODE HERE
            raise NotImplementedError()
        return x

In [ ]:
# Run the sanity check
run_tests(Dropout=Dropout)

<a id="chp5-optimization"></a>
# 5. Optimizers
There are many different optimization algorithms that can be used to train a neural network, such as stochastic gradient descent (SGD), RMSprop, Adam, AdamW, etc. In this exercise, we will compare two popular optimizers: SGD with momentum and the AdamW optimizer. Both have proven to work well in a large variety of different settings; in particular AdamW is a sensible default choice, yielding good performance with little hyperparameter tuning.

## 5.1 SGD with momentum and weight decay

The SGD optimizer with momentum and L2 regularization updates the parameters $\mathbf{w}$ of the model using the following formulas:

$$
\mathbf{v}_t = \beta \mathbf{v}_{t-1} + \left( \cfrac{\partial \text{E}}{\partial \mathbf{w}_{t-1}} + \lambda \mathbf{w}_{t-1} \right)
$$
$$
\mathbf{w}_t = \mathbf{w}_{t-1} - \eta \mathbf{v}_t
$$
where $\mathbf{w}_t$ is the updated parameter, $\mathbf{w}_{t-1}$ is the previous parameter, $\eta$ is the learning rate, $\beta$ is the momentum, $\cfrac{\partial \text{E}}{\partial \mathbf{w}_{t-1}}$ is the gradient of the loss function with respect to the parameter, and $\lambda$ is the regularization strength (L2 penalty coefficient).

**Notes:**
- $\beta = 0$ yields the regular SGD update (no momentum).
- L2 regularization is equivalent to weight decay _only without momentum_, that is in case $\beta = 0$. When using momentum (or other optimizers), L2 regularization and weight decay are not equivalent. However, this distinction is not always made in the literature and popular frameworks, so be wary!
- Weight decay downscales all weights by a small factor in each step (hence the name). This is not appropriate for all parameters in a network! In particular, avoid penalizing biases and affine parameters of normalization layers.

Have a good look at the following example implementation of SGD with momentum and weight decay.
PyTorch's optimizer base class is very flexible - we're using only a small part of it. For a more comprehensive overview, check the [documentation](https://pytorch.org/docs/stable/optim.html).

In [ ]:
class SGD(torch.optim.Optimizer):
    def __init__(self, params, lr: float, momentum: float = 0.0, weight_decay: float = 0.0):
        # each parameter group is a dictionary containing the parameters and hyperparameters
        defaults = dict(lr=lr, momentum=momentum, weight_decay=weight_decay)
        super().__init__(params, defaults)
        torch.optim.SGD

    @torch.no_grad()
    def step(self):
        # self.param_groups is a list of dictionaries
        for group in self.param_groups:
            for param in group['params']:
                if param.grad is None:
                    continue
                    
                grad = param.grad
                # the optimizer base class manages the state for each parameter
                state = self.state[param]
                
                # Add weight decay to gradient
                if group['weight_decay'] != 0:
                    grad = grad + group['weight_decay'] * param
                
                # Apply momentum
                if group['momentum'] != 0:
                    if 'momentum_buffer' not in state:
                        state['momentum_buffer'] = torch.clone(grad).detach()
                    else:
                        # suffix _ means inplace operation
                        state['momentum_buffer'].mul_(group['momentum']).add_(grad)
                        # equivalent to m = beta * m + grad
                    grad = state['momentum_buffer']
                
                # SGD update
                param.add_(grad, alpha=-group['lr'])

## 5.2 AdamW

AdamW combines Adam's adaptive learning rates with decoupled weight decay. For each parameter vector $\mathbf{w}$, it maintains exponential moving averages of gradients ($\mathbf{m}$) and squared gradients ($\mathbf{v}$):

**Moment Updates:**
$$\mathbf{m}_t = \beta_1 \mathbf{m}_{t-1} + (1-\beta_1) \mathbf{g}_t$$
$$\mathbf{v}_t = \beta_2 \mathbf{v}_{t-1} + (1-\beta_2) \mathbf{g}_t^2$$

**Bias Correction:**
$$\hat{\mathbf{m}}_t = \mathbf{m}_t/(1-\beta_1^t)$$
$$\hat{\mathbf{v}}_t = \mathbf{v}_t/(1-\beta_2^t)$$

**Parameter Update:**
$$\mathbf{w}_t = (1-\eta\lambda)\mathbf{w}_{t-1} - \eta\frac{\hat{\mathbf{m}}_t}{(\sqrt{\hat{\mathbf{v}}_t}+\epsilon)}$$

where $\beta_1=0.9$, $\beta_2=0.999$ are the decay rates, $\epsilon$ is a small constant for numerical stability, and $\lambda$ is the weight decay coefficient. The decoupled weight decay term $(1-\eta\lambda)\mathbf{w}_{t-1}$ distinguishes AdamW from standard Adam. Note that with this formulation, a learning rate scheduler also influences the weight decay rate!

**TODO:** implement the update equations of the AdamW optimizer.

In [ ]:
class AdamW(torch.optim.Optimizer):
    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0.0):
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        super().__init__(params, defaults)

    @torch.no_grad()
    def step(self):
        for group in self.param_groups:
            beta1, beta2 = group['betas']
            
            for param in group['params']:
                if param.grad is None:
                    continue
                    
                grad = param.grad
                state = self.state[param]
                
                # State initialization
                if len(state) == 0:
                    state['step'] = 0
                    # Exponential moving average of gradient values
                    state['exp_avg'] = torch.zeros_like(param)
                    # Exponential moving average of squared gradient values
                    state['exp_avg_sq'] = torch.zeros_like(param)
                
                state['step'] += 1
                
                # Decay the first and second moment running average coefficient
                # YOUR CODE HERE
                raise NotImplementedError()
                
                # Bias correction - cheaper to precompute it
                # YOUR CODE HERE
                raise NotImplementedError()
                
                # AdamW = Adam + decoupled weight decay
                # YOUR CODE HERE
                raise NotImplementedError()
                

In [ ]:
# Run the sanity check
run_tests(optimizer=AdamW)

<a id="chp6-experiments"></a>
# 6. Experiments

In this section, we will conduct a series of experiments to evaluate the performance of the neural network with/without layer normalization, dropout, and different optimization algorithms. We will use the [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) dataset, which has the same size and numbers of images as the original MNIST dataset, but with more challenging images of clothing items.

**All experiments log data via TensorBoard.**

Open TensorBoard by running the following command in the terminal:
```
tensorboard --logdir=runs
```
Run the code below to train the neural network with different configurations and analyze the results.

## 6.1 SGD vs AdamW
In this section, we will compare the performance of the SGD and AdamW optimizers on the Fashion MNIST dataset by training a simple neural network.

After training the models, what do you observe when comparing the `train_loss` curves? Which of the two gives a better accuracy, and which one converges faster? Can you tune them to give better results?

In [ ]:
optimizer_config = ExperimentConfig(
    exp_name="Optimizer",
    config_name=None,
    use_layernorm=False,
    dropout_rate=0,
    max_steps=3000,
    device="cpu",
)

sgd_config = replace(
    optimizer_config,
    config_name="SGD",
    optimizer="sgd",
    lr=1e-2,
)
sgd_trainer = Trainer(sgd_config)
sgd_trainer.run_experiment()

In [ ]:
adamw_config = replace(
    optimizer_config,
    config_name="AdamW",
    optimizer="adamw",
    lr=3e-4,
)
adamw_trainer = Trainer(adamw_config)
adamw_trainer.run_experiment()

## 6.2 Layer Normalization and Dropout
In this experiment, we will investigate the effects of Layer Normalization and Dropout on the performance of a neural network. We will train the model under four different configurations: 
1. Without Layer Normalization and Dropout
2. With only Layer Normalization
3. With only Dropout
4. With both Layer Normalization and Dropout. 

What do you observe when analyzing the training and validation curves? Which configuration performs best, and what do you conclude about the benefits of using DropOut and/or LayerNorm?

You can also play around and try to get better results - try different optimizers and hyperparameters.

In [ ]:
layernorm_dropout_config = ExperimentConfig(
    exp_name="LayernormDropout",
    config_name=None,
)

base_config = replace(
    layernorm_dropout_config,
    config_name="no_ln_no_dropout",
    use_layernorm=False,
    dropout_rate=0,
)

pprint(base_config)
base_trainer = Trainer(base_config)
base_trainer.run_experiment()

In [ ]:
ln_config = replace(
    layernorm_dropout_config,
    config_name="layernorm_only",
    use_layernorm=True,
    dropout_rate=0,
)
ln_config_trainer = Trainer(ln_config)
ln_config_trainer.run_experiment()

In [ ]:
dropout_config = replace(
    layernorm_dropout_config,
    config_name="dropout_only",
    use_layernorm=False,
    dropout_rate=0.15,
)
dropout_trainer = Trainer(dropout_config)
dropout_trainer.run_experiment()

In [ ]:
ln_dropout_config = replace(
    layernorm_dropout_config,
    config_name="layernorm_and_dropout",
    use_layernorm=True,
    dropout_rate=0.15,
)
ln_dropout_trainer = Trainer(ln_dropout_config)
ln_dropout_trainer.run_experiment()